In [21]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [22]:
!pip install opencv-python==3.4.2.17
!pip install opencv-contrib-python==3.4.2.17

import numpy as np
import cv2 as cv
from google.colab.patches import cv2_imshow
import copy
from copy import deepcopy
import os
import sklearn
import keras
import tensorflow
from sklearn.model_selection import train_test_split
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.utils import to_categorical
import random
import string

In [23]:
!python --version
print('Numpy version: ', np.__version__)
print('OpenCV2 version: ', cv.__version__)
print('Sklearn version: ', sklearn.__version__)
print('Keras version: ', keras.__version__)
print('Tensorflow version: ', tensorflow.__version__)

Python 3.7.13
Numpy version:  1.21.6
OpenCV2 version:  3.4.2
Sklearn version:  1.0.2
Keras version:  2.8.0
Tensorflow version:  2.8.0


In [24]:
BASE_FOLDER = 'test'  # this defines the folder with the input data; change to test for testing file 

# Board definition.

For the board, we will define a template that encodes the initial information in each patch (1 -> triple word, 2 -> double word, 3 -> triple letter, 4 -> double letter, 0 -> otherwise). Each board will also contain the letters or replacement letter (the empty box) once they are extracted from each image. The replacement letter will be denoted by '-' on the board representation.

In [25]:
# the dictionary of the values of each letter
letter_score = {'A':1, 'B':3, 'C':3, 'D':2, 'E':1, 'F':4, 'G':2, 'H':4, 'I':1, 'J': 8, 'K':5, 'L':1, 'M':3, 'N':1, 'O':1, 'P':3, 'Q':10, 'R':1, 'S':1, 'T':1, 'U':1, 'V':4, 'W':4, 'X':8, 'Y':4, 'Z':10, '-':0}

In [26]:
NO_SQUARES = 15  # number of squares on the board (on each row/col)

def no_to_letter(indx):
    """
    The mapping from indices to letters for the column letter representation (0->a, 1->b, 2->c etc)
    """
    return chr(ord('`') + indx + 1)

In [27]:
# the positions of the premium squares
prem_sq_positions = {'triple-word' : [(0, 0), (0, 7), (0, 14), (7, 0), (7, 14), (14, 0), (14, 7), (14, 14)], 'double-word' : [(1, 1), (2, 2), (3, 3), (4, 4), (1, 13), (2, 12), (3, 11), (4, 10), (7, 7), (10, 4), (11, 3), (12, 2), (13, 1), (10, 10), (11, 11), (12, 12), (13, 13)], 'triple-letter' : [(1, 5), (1, 9), (5, 5), (5, 9), (9, 5), (9, 9), (13, 5), (13, 9), (5, 1), (9, 1), (5, 13), (9, 13)], 'double-letter': [(0, 3), (0, 11), (2, 6), (2, 8), (3, 0), (3, 7), (3, 14), (6, 2), (6, 6), (6, 8), (6, 12), (7, 3), (7, 11), (8, 2), (8, 6), (8, 8), (8, 12), (11, 0), (11, 7), (11, 14), (12, 6), (12, 8), (14, 3), (14, 11)]}

In [28]:
# the premium square name mapped to a number for the template board (1 -> triple word, 2 -> double word, 3 -> triple letter, 4 -> double letter)
sq_to_number = {'triple-word' : 1, 'double-word' : 2, 'triple-letter' : 3, 'double-letter': 4}

In [29]:
def get_board_template():
    """
    The template for the board (1 -> triple word, 2 -> double word, 3 -> triple letter, 4 -> double letter, 0 otherwise)
    """
    board_template = [[0 for _ in range(NO_SQUARES)] for _ in range(NO_SQUARES)]
    for i in range(NO_SQUARES):
        for j in range(NO_SQUARES):
            for prem_sq in prem_sq_positions:
                if (i, j) in prem_sq_positions[prem_sq]:
                    board_template[i][j] = sq_to_number[prem_sq]
    return board_template

In [30]:
board_template = get_board_template()

# Scoring computations

In order to compute the score of each board, we will need the previous board (which initially is the empty template) and the current board. Then, we will extarct all the new letters from the current board and filter them so that we do not have isolated letters that are the result of an error in the letter recognition model (more about it below). Then, we will check if there are any replacement letters identified and again filter them.
We will then extract each vertical and horizontal word from both current and previous boards and then, for the new words, we will compute the score.

In [31]:
def are_neigh_around(new_letters_temp, i1, j1):
    """
    It checks if the current letter have any neighbours around it or is isolated.
    If it is isolated it is likely to be an error from the CNN which recognize the letters.

    Parameters
    ----------
    new_letters_temp : list
        The list of new letters on the board.
    i1 : int
        The row of the word    
    j1 : int
        The col of the word

    Returns
    -------
    have_neigh: bool
       True if it has heigbors (maximum distance between letters is 2).
    """
    have_neigh = False
    for [l, i, j] in new_letters_temp:
        if i == i1 and j == j1:
            continue
        if abs(i - i1) < 2 or abs(j - j1) < 2:
            have_neigh = True
    return have_neigh

In [32]:
def remove_false_new_letters(new_letters, new_letters_temp):
    """
    It checks if there are isolated letters whcich are likely to be an error from the CNN which recognize the letters.

    Parameters
    ----------
    new_letters : list
        The list of new letters on the board and their coordinates as they should be outputed.
    new_letters_temp : list
        The list of new letters on the board and their coordinates in the matrix representig the board.
    Returns
    -------
    new_letters_real, new_letters_temp_real: list, list
       The letters that are not isolated
    """
    new_letters_real = []
    new_letters_temp_real = []
    i_set = set()
    i_dict = {}
    j_set = set()
    j_dict = {}
    for [l, i, j] in new_letters_temp:
        if i in i_dict:
            i_dict[i] += 1
        else:
            i_dict[i] = 0
        if j in j_dict:
            j_dict[j] += 1
        else:
            j_dict[j] = 0
        i_set.add(i)
        j_set.add(j)
    if len(i_set) < len(j_set):  # horizontal word
        max_key = max(i_dict, key=i_dict.get)  # the most common row
        for indx, [l, i, j] in enumerate(new_letters_temp):
            if i != max_key:  # if the letter is not on the same row with the newly added word, it must be an outsider
                continue
            if not are_neigh_around(new_letters_temp, i, j):  # if the letter has no neighbors, it must be an outsider
                continue
            new_letters_temp_real.append([l, i, j])
            new_letters_real.append(new_letters[indx])
    else: # vertical word
        max_key = max(j_dict, key=j_dict.get)  # the most common col
        for indx, [l, i, j] in enumerate(new_letters_temp):
            if j != max_key:  # if the letter is not on the same col with the newly added word, it must be an outsider
                continue
            if not are_neigh_around(new_letters_temp, i, j):  # if the letter has no neighbors, it must be an outsider
                continue
            new_letters_temp_real.append([l, i, j])
            new_letters_real.append(new_letters[indx])
    return new_letters_real, new_letters_temp_real

In [33]:
def board_score(prev_board, curr_board):
    """
    Computes the score of the board given the current board and the previous one.
    Also identifies the new letters added on the board.

    Parameters
    ----------
    prev_board : list
        The board in the previous round.
    curr_board : list
        The board in the current round.

    Returns
    -------
    score: int
       The score of the board
    new_letters: list
      The list of the new letters and their possition
    """
    no_letter_info = [] # the replacements for the letter found on the table (all '?' found)
    new_letters = []
    new_letters_temp = []
    score = 0
    for i in range(NO_SQUARES):
        for j in range(NO_SQUARES):
            # check if a letter is new on the board (is not present on the previous board)
            if isinstance(curr_board[i][j], str) and curr_board[i][j] != prev_board[i][j] and curr_board[i][j] != '-':
                new_letters.append([curr_board[i][j], i + 1, no_to_letter(j).upper()])
                new_letters_temp.append([curr_board[i][j], i, j])
            if isinstance(curr_board[i][j], str) and curr_board[i][j] == '-':
                no_letter_info.append(['?', i, j])
    new_letters, new_letters_temp = remove_false_new_letters(new_letters, new_letters_temp)
    if len(new_letters) == 7:  # if all 7 letters have been used, add the bonus score
        score += 50

    # add '?' when the letter replacement is found, and make sure that the '?' found is not a recognition error
    no_letter_info_final = []  # the real '?' added on the table if it exists
    if len(no_letter_info) > 0:
        if len(new_letters_temp) >= 2 and new_letters_temp[0][1] == new_letters_temp[1][1]:  # horizontal new word
            for [l, i, j] in no_letter_info:
                if i != new_letters_temp[0][1]:  # if the replacement is not on the same row with the newly added word, it must be an outsider
                    continue
                if not are_neigh_around(new_letters_temp, i, j):  # if the replacement has no neighbors, it must be an outsider
                    continue
                no_letter_info_final = ['?', i, j]
            if no_letter_info_final != []:  # find the replacement that is near or between one of the newly added letters
                for index_list in range(len(new_letters_temp) - 1):
                    if new_letters_temp[index_list][2] < no_letter_info_final[2] and new_letters_temp[index_list+1][2] > no_letter_info_final[2]:
                        new_letters_temp.insert(index_list, no_letter_info_final)
                        new_letters.insert(index_list, [no_letter_info_final[0], no_letter_info_final[1] + 1, no_to_letter(no_letter_info_final[2]).upper()])
    if len(no_letter_info) > 0:
        if len(new_letters_temp) >= 2 and new_letters_temp[0][2] == new_letters_temp[1][2]:  # vertical new word
            for [l, i, j] in no_letter_info:
                if j != new_letters_temp[0][2]:    # if the replacement is not on the same col with the newly added word, it must be an outsider
                    continue
                if not are_neigh_around(new_letters_temp, i, j):  # if the replacement has no neighbors, it must be an outsider
                    continue
                no_letter_info_final = ['?', i, j]
            if no_letter_info_final != []:  # find the replacement that is near or between one of the newly added letters
                for index_list in range(len(new_letters_temp) - 1):
                    if new_letters_temp[index_list][1] < no_letter_info_final[1] and new_letters_temp[index_list+1][1] > no_letter_info_final[1]:
                        new_letters_temp.insert(index_list, no_letter_info_final)
                        new_letters.insert(index_list, [no_letter_info_final[0], no_letter_info_final[1] + 1, no_to_letter(no_letter_info_final[2]).upper()])    

    prev_vertical_words = []  # the words and their starting position that can be found on the vertical axes of the previous board
    prev_horizontal_words = []  # the words and their starting position that can be found on the horizontal axes of the previous board
    curr_vertical_words = []  # the words and their starting position that can be found on the vertical axes of the current board
    curr_horizontal_words = []  # the words and their starting position that can be found on the horizontal axes of the current board

    # compute the words and their starting position that can be found on the horizontal axes of the previous board
    for i in range(NO_SQUARES):
        j = 0
        while j < NO_SQUARES:
            if isinstance(prev_board[i][j], str):
                j_start = j
                j += 1
                word = ''
                word += prev_board[i][j_start]
                while j < NO_SQUARES and (isinstance(prev_board[i][j], str) or prev_board[i][j] == '-'):
                    word += prev_board[i][j]
                    j += 1
                if len(word) == 1:
                    continue
                else:
                    prev_horizontal_words.append([word, i, j_start])
            else:
                  j += 1
    # compute the words and their starting position that can be found on the horizontal axes of the current board  
    for i in range(NO_SQUARES):
        j = 0
        while j < NO_SQUARES:
            if isinstance(curr_board[i][j], str):
                j_start = j
                j += 1
                word = ''
                word += curr_board[i][j_start]
                while j < NO_SQUARES and (isinstance(curr_board[i][j], str) or curr_board[i][j] == '-'):
                    word += curr_board[i][j]
                    j += 1
                if len(word) == 1:
                    continue
                else:
                    curr_horizontal_words.append([word, i, j_start])
            else:
                  j += 1
    # compute the words and their starting position that can be found on the vertical axes of the previous board
    for i in range(NO_SQUARES):
        j = 0
        while j < NO_SQUARES:
            if isinstance(prev_board[j][i], str):
                j_start = j
                j += 1
                word = ''
                word += prev_board[j_start][i]
                while j < NO_SQUARES and (isinstance(prev_board[j][i], str) or prev_board[j][i] == '-'):
                    word += prev_board[j][i]
                    j += 1
                if len(word) == 1:
                    continue
                else:
                    prev_vertical_words.append([word, j_start, i])
            else:
                  j += 1
    # compute the words and their starting position that can be found on the vertical axes of the current board
    for i in range(NO_SQUARES):
        j = 0
        while j < NO_SQUARES:
            if isinstance(curr_board[j][i], str):
                j_start = j
                j += 1
                word = ''
                word += curr_board[j_start][i]
                while j < NO_SQUARES and (isinstance(curr_board[j][i], str) or curr_board[j][i] == '-'):
                    word += curr_board[j][i]
                    j += 1
                if len(word) == 1:
                    continue
                else:
                    curr_vertical_words.append([word, j_start, i])
            else:
                  j += 1
    # for each new horizontal word, compute the score
    for [word, i, j] in curr_horizontal_words:
        if [word, i, j] in prev_horizontal_words:
            continue
        s1 = 0
        tr_word = 1
        db_word = 1
        for letter in word: # (1 -> triple word, 2 -> double word, 3 -> triple letter, 4 -> double letter, 0 otherwise)
            if prev_board[i][j] == 1:
                tr_word *= 3
            if prev_board[i][j] == 2:
                db_word *= 2
            if prev_board[i][j] == 3:
                s1 += letter_score[letter] * 3
            elif prev_board[i][j] == 4:
                s1 += letter_score[letter] * 2
            else: 
                s1 += letter_score[letter]
            j += 1
        s1 *= db_word * tr_word
        score += s1  # add the score of this word to the global score of this round

    # for each new vertical word, compute the score
    for [word, i, j] in curr_vertical_words:
        if [word, i, j] in prev_vertical_words:
            continue
        s2 = 0
        tr_word = 1
        db_word = 1
        for letter in word: # (1 -> triple word, 2 -> double word, 3 -> triple letter, 4 -> double letter, 0 otherwise)
            if prev_board[i][j] == 1:
                tr_word *= 3
            if prev_board[i][j] == 2:
                db_word *= 2
            if prev_board[i][j] == 3:
                s2 += letter_score[letter] * 3
            elif prev_board[i][j] == 4:
                s2 += letter_score[letter] * 2
            else:
                s2 += letter_score[letter]
            i += 1
        s2 *= db_word * tr_word
        score += s2  # add the score of this word to the global score of this round

    return score, new_letters

Helper functions for visualizing the output of the template matching and patch identification. These functions are not used in the final form of this notebook.

In [34]:
def draw_lines(image, lines, color: tuple = (0, 0, 255)):
    """
    Draws the lines on the image.

    Parameters
    ----------
    image : numpy array
        The image we want to draw lines on
    lines : list
        The list of lines
    color : tuple
        The color of the lines
    """
    drawing = image.copy()
    if drawing.ndim == 2:
        drawing = cv.cvtColor(drawing, cv.COLOR_GRAY2BGR)
    for line in lines: 
        cv.line(drawing, line[0], line[1], color, 2, cv.LINE_AA)
        
    cv2_imshow(cv.resize(drawing, (0, 0), fx=0.15, fy=0.15))

In [35]:
def draw_patches(image, patches):
    """
    Draws the patches on the image.

    Parameters
    ----------
    image : numpy array
        The image we want to draw lines on
    patches : list
        The list of all patches
    """
    patches_lines = []
    for patch in patches:
        [(x1, y1), (x2, y2)] = patch
        r1 = [[(x1, y1), (x2, y1)]]
        r2 = [[(x1, y2), (x2, y2)]]
        c1 = [[(x1, y1), (x1, y2)]]
        c2 = [[(x2, y1), (x2, y2)]]
        patches_lines += r1 + r2 + c1 + c2
    draw_lines(image, patches_lines)

In [36]:
def draw_patch(image, patch):
    """
    Draws a single patch on the image.

    Parameters
    ----------
    image : numpy array
        The image we want to draw lines on
    patch : list
        The list coordinates for the patch to be drawn
    """
    [(x1, y1), (x2, y2)] = patch
    r1 = [[(x1, y1), (x2, y1)]]
    r2 = [[(x1, y2), (x2, y2)]]
    c1 = [[(x1, y1), (x1, y2)]]
    c2 = [[(x2, y1), (x2, y2)]]
    draw_lines(image, r1+r2+c1+c2)

Manual identification of all the different patches from a chosen template. The function returns the coordinates of each patch.

In [37]:
def get_all_patches_from_template(padd_x=0, padd_y=0):
    """
    Computes the patches for the template. The rows and columns coordinates are hardcoded.

    Parameters
    ----------

    Returns
    -------
    patches: list
       A list of all the patches (top-left corner and bottom-right corner).
    """

    l = [
    [(0, 40), (2172, 40)],
    [(0, 190), (2172, 190)],
    [(0, 340), (2172, 340)],
    [(0, 490), (2172, 490)],
    [(0, 640), (2172, 640)],
    [(0, 790), (2172, 790)],
    [(0, 940), (2172, 940)],
    [(0, 1090), (2172, 1090)],
    [(0, 1240), (2172, 1240)],
    [(0, 1390), (2172, 1390)],
    [(0, 1540), (2172, 1540)],
    [(0, 1690), (2172, 1690)],
    [(0, 1840), (2172, 1840)],
    [(0, 1990), (2172, 1990)],
    [(0, 2140), (2172, 2140)],
    [(0, 2290), (2172, 2290)]]

    c = [
    [(40, 0), (40, 2305)],
    [(180, 0), (180, 2305)],
    [(320, 0), (320, 2305)],
    [(460, 0), (460, 2305)],
    [(600, 0), (600, 2305)],
    [(740, 0), (740, 2305)],
    [(880, 0), (880, 2305)],
    [(1020, 0), (1020, 2305)],
    [(1160, 0), (1160, 2305)],
    [(1300, 0), (1300, 2305)],
    [(1440, 0), (1440, 2305)],
    [(1580, 0), (1580, 2305)],
    [(1720, 0), (1720, 2305)],
    [(1860, 0), (1860, 2305)],
    [(2000, 0), (2000, 2305)],
    [(2140, 0), (2140, 2305)]]

    patches = []
    for i in range(0, len(l) - 1):
        for j in range(0, len(c) - 1):
            [(x1, y1), (_, _)] = l[i]
            [(x2, y2), (_, _)] = c[j]
            [(x3, y3), (_, _)] = l[i+1]
            [(x4, y4), (_, _)] = c[j+1]
            patches.append([(x2+padd_x, y1+padd_y), (x4+padd_x, y3+padd_y)])

    return patches

# SIFT

The following functions are addapted from laboratory 4 and use SIFT features to first align the input image to the template and then to crop the input aligned image according to the template (so that we can use the patches identified before for each input image).

In [38]:
def get_keypoints_and_features(image) -> tuple:
    """
    Creates the SIFT object and find the keypoints from the grayscale image. Compute the features based on the grayscale image and the keypoints.
    The code is addapted from Laboratory 4.
    
    Parameters
    ----------
      image: numpy array
          The image to be processed

    Returns
    -------
      keypoints: [cv.Keypoint] 
          The keypoints of the image
      features: np.ndarray
          The features for each keypoint.
    """
    gray_image = cv.cvtColor(image, cv.COLOR_BGR2GRAY) 
    
    sift = cv.xfeatures2d.SIFT_create()
    keypoints = sift.detect(gray_image, None)
    keypoints, features = sift.compute(gray_image, keypoints) 
        
    return keypoints, features

In [39]:
def match_features(features_source, features_dest) -> [[cv.DMatch]]:
    """
    Match features from the source image with the features from the destination image.
    The code is addapted from Laboratory 4.
    
    Parameters
    ----------
      features_source: np.ndarray
          The features from the source image
      features_dest: np.ndarray
          The features from the destination image

    Returns
    -------
      matches: [[DMatch]]
          The rusult of the matching. For each set of features from the source image,
          it returns the first 'K' matchings from the destination images.
    """
     
    feature_matcher = cv.DescriptorMatcher_create("FlannBased")
    matches = feature_matcher.knnMatch(features_source, features_dest, k=2)   
    return matches

In [40]:
def generate_homography(all_matches:  [cv.DMatch], keypoints_source: [cv.KeyPoint], keypoints_dest : [cv.KeyPoint],
                        ratio: float = 0.75, ransac_rep: int = 4.0):
    """
    Find the matchings that pass the Lowe's ratio test (ratio parameter).
    Get the coordinates of the keypoints from the source image.
    Get the coordinates of the keypoints from the destination image.
    Obtain the Homagraphy.
    The code is addapted from Laboratory 4.

    Parameters
    ----------
    all_matches : [DMatch]
    keypoints_source : [cv.Point]
    ratio : float
      Lowe's ratio test (the ratio 1st neighbour distance / 2nd neighbour distance)
    keypoints_source: nd.array [Nx2] (x, y coordinates)
    keypoints_dest: nd.array [Nx2] (x, y coordinates)
    ransac_rep: float
      The threshold in the RANSAC algorithm.

    Returns
    -------
    H
      The homography matrix.
    """
    if not all_matches:
        return None
    
    matches = [] 
    for match in all_matches:  
        if len(match) == 2 and (match[0].distance / match[1].distance) < ratio:
            matches.append(match[0])
     
    points_source = np.float32([keypoints_source[m.queryIdx].pt for m in matches])
    points_dest = np.float32([keypoints_dest[m.trainIdx].pt for m in matches])

    if len(points_source) > 4:
        H, status = cv.findHomography(points_source, points_dest, cv.RANSAC, ransac_rep)
        return H
    else:
        return None

In [41]:
def pad_image(image_, procent=0.1):
    """
    Padds an image.
    The code is addapted from Laboratory 4.
    
    Parameters
    ----------
      image_: np.ndarray
          The image we want to padd
      procent: float
          The percentage to add as padding.

    Returns
    -------
      big_image: np.ndarray
          The padded image
    """
    pad_h = int(image_.shape[0] * procent)
    pad_w = int(image_.shape[1] * procent)
    big_image = np.zeros((image_.shape[0] + 2 * pad_h, image_.shape[1] + pad_w, 3), np.uint8)
    big_image[pad_h: pad_h + image_.shape[0], pad_w: pad_w + image_.shape[1]] = image_.copy()
    return big_image

In [49]:
def stitch_images_inside(image_source, image_dest):
    """
    Get the keypoints and the features from the source image.
    Get the keypoints and the features from the destination image.
    Match the features.
    Find the homography matrix.
    Apply the homography matrix on the source image.
    Copy the destination image in the resulting image from the previous point, but keep the resulting pixels in place!
    The code is addapted from Laboratory 4.
    
    Parameters
    ----------
      image_source: np.ndarray
          The source image
      image_source: np.ndarray
          The destination image

    Returns
    -------
      result: np.ndarray
          The stitched image
    """
 
    image_dest = pad_image(image_dest)
    keypoints_source, features_source = get_keypoints_and_features(image_source)
    keypoints_dest, features_dest = get_keypoints_and_features(image_dest)
          
    all_matches = match_features(features_source, features_dest)
        
    H = generate_homography(all_matches, keypoints_source, keypoints_dest, 0.75, 4)

    result = cv.warpPerspective(image_source, H, 
        (image_dest.shape[1], image_dest.shape[0]))
    
    mask = result[:, :] == [0, 0, 0]
    result = result * (1 - mask) + image_dest * mask
    return result.astype(np.uint8)
        

In [51]:
def crop_image_as_template(image_source, image_dest):
    """
    Get the keypoints and the features from the source image.
    Get the keypoints and the features from the destination image.
    Match the features.
    Find the homography matrix.
    Apply the homography matrix on the source image.
    Crop the destination image according to the source image.
    The code is addapted from Laboratory 4.
    
    Parameters
    ----------
      image_source: np.ndarray
          The source image
      image_source: np.ndarray
          The destination image

    Returns
    -------
      result: np.ndarray
          The stitched image
    """
 
    image_dest = pad_image(image_dest)
    keypoints_source, features_source = get_keypoints_and_features(image_source)
    keypoints_dest, features_dest = get_keypoints_and_features(image_dest)
          
    all_matches = match_features(features_source, features_dest)
        
    H = generate_homography(all_matches, keypoints_source, keypoints_dest, 0.75, 4)

    result = cv.warpPerspective(image_source, H, 
        (image_dest.shape[1], image_dest.shape[0]))
    
    mask = result[:, :] == [0, 0, 0]

    y, x, _ = np.nonzero(result) # crop the black edge
    
    return image_dest[np.min(y):np.max(y), np.min(x):np.max(x)]

# CNN model

Generate image patches to train a CNN model to classify them.

For each train image, I have alligned it to the template, cropped it according to the template and then I have generated all the patches, using the function below. Each patch has been saved in a folder named 'new_images' and then used to train a model that classifies these patches.

In [44]:
def generate_cropped_images(image, patches, img_nr):
    """
    Generate all images from all patches.

    Parameters
    ----------
    image : numpy array
        The image to be processed
    patches : list
        The list of all patches

    Returns
    -------
    """
    if not os.path.exists('/content/gdrive/MyDrive/new_images'):
        os.makedirs('/content/gdrive/MyDrive/new_images')
    indx = -1
    for i in range(NO_SQUARES):
        for j in range(NO_SQUARES):
            indx += 1
            [(x1, y1), (x2, y2)] = patches[indx]
            cropped_image = image[y1:y2, x1:x2]
            cv.imwrite('/content/gdrive/MyDrive/new_images/' + 'img_' + str(img_nr) + '_' + str(indx) + '_' + str(i) + '_' + str(j) + '.jpg', cropped_image)


In [47]:
# read all images from train and add them in images_train array
base_folder = 'train'
image_names = os.listdir(base_folder)
image_names.sort()
num_images = len(image_names) // 2
images_train = []
for img in image_names:
    if img[-3:] == 'jpg':
        image_now = cv.imread(os.path.join(base_folder, img))
        images_train.append(image_now)

In [53]:
# transform each image so that it maches a template, then gennerate all patches (uncomment to do so)
# img_indx = -1
# for img in images_train:
#     img_indx += 1
#     image_now = cv.imread('template.jpg')
#     image_then = img
#     stitched = stitch_images_inside(image_source=image_then, image_dest=image_now)
#     image_then = cv.imread('template.jpg')
#     image_now = stitched.astype(np.uint8)
#     stitched2 = crop_image_as_template(image_source=image_then, image_dest=image_now)
#     generate_cropped_images(stitched2, get_all_patches_from_template(335, 155), img_indx)

# CNN model

Based on the images that have been extracted previously (all the patches), I have manually classified a part of them and then trained a CNN to recognize each patch. The data used for training this CNN is located in Custom_Data.zip.

There are 29 classes: one for each letter, one for the replacement piece, one for the empty patch and one for the paches that contain double/triple the score.

The source of inspiration for the architecture of the CNN model was https://www.analyticsvidhya.com/blog/2020/02/learn-image-classification-cnn-convolutional-neural-networks-3-datasets/ which classifies the mnist dataset.

Therefore, the architecture consists of 2 2D convolutional layers (one with 30 filters and one with 60 and a kernel of 3x3), followed by a max pooling layer of 2x2 and a dropout of 20%. Then another 2D convolutional layer of 120 filters followed by a max pooling layer and a dropout of 20%. Then the output is flattened and added into a dense layer (densely-connected NN) of 250 neurons, followed by a 30% dropout and another dense layer with softmax activation that does the final classification.

The optimizer used for the model is Adam and the loss function is categorical_crossentropy as the labels are transformed into categorical data. The preprocessing for the input data is: the images are transformed into black and white images and then 3 gaussian blur filters are applied. The data is then reshaped and normalized and feeded into the model.

The model was previously run and saved into MyBestModel2.zip so that we don't have to run it each time (the running time of the model is approximately 2h 30m. If it needs to be run again, the following code must be uncommented.

In [54]:
# !unzip '/content/Custom_Data.zip'

In [55]:
# dictionaries used to decode and encode the labels of the data
cat_to_label = {'A':1,'B':2,'C':3,'D':4,'E':5,'F':6,'G':7,'H':8,'I':9,'J':10,'K':11,'L':12,'M':13,'N':14,'O':15,'P':16,'Q':17,'R':18,'S':19,'T':20,'U':21,'V':22,'W':23,'X':24,'Y':25,'Z':26,'Nothing':27,'Double':28,'NoLetter':29}
label_to_cat = {1:'A',2:'B',3:'C',4:'D',5:'E',6:'F',7:'G',8:'H',9:'I',10:'J',11:'K',12:'L',13:'M',14:'N',15:'O',16:'P',17:'Q',18:'R',19:'S',20:'T',21:'U',22:'V',23:'W',24:'X',25:'Y',26:'Z',27:'Nothing',28:'Double',29:'NoLetter'}

In [56]:
# train_letter_data = {x : [] for x in ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','Nothing','Double','NoLetter']}
# base_folder = 'Custom_Data'
# folder_names = os.listdir(base_folder)
# for cat in folder_names:
#     image_names = os.listdir(base_folder + '/' + cat)
#     for img in image_names:
#         if img[-3:] == 'jpg':
#             my_img = cv.imread(os.path.join(base_folder, cat, img))
#             my_img = cv.cvtColor(my_img, cv.COLOR_BGR2GRAY)
#             my_img = cv.GaussianBlur(my_img, (3, 3), 0)
#             my_img = cv.GaussianBlur(my_img, (3, 3), 0)
#             my_img = cv.GaussianBlur(my_img, (3, 3), 0)
#             train_letter_data[cat].append(my_img)

In [57]:
# all_train_letter_data = []
# all_train_letter_labels = []
# for cat in train_letter_data:
#     for img in train_letter_data[cat]:
#         all_train_letter_data.append(img)
#         all_train_letter_labels.append(cat_to_label[cat])

In [58]:
# X_train = np.array(all_train_letter_data)
# Y_train = np.array(all_train_letter_labels)

In [63]:
# X_train = X_train.reshape(X_train.shape[0], 150, 140, 1)  # reshape data
# X_train = X_train.astype('float32')
# X_train /= 255  # normalize data
# y_train = to_categorical(Y_train)  # transform the labeling into categorical

# # construct the model
# model = Sequential()
# model.add(Conv2D(30, kernel_size=(3,3), padding='same', activation='relu', input_shape=(150, 140, 1)))
# model.add(Conv2D(60, kernel_size=(3,3), padding='same', activation='relu'))
# model.add(MaxPool2D(pool_size=(2,2)))
# model.add(Dropout(0.2))
# model.add(Conv2D(120, kernel_size=(3,3), padding='same', activation='relu'))
# model.add(MaxPool2D(pool_size=(2,2)))
# model.add(Dropout(0.2))
# model.add(Flatten())
# model.add(Dense(250, activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(30, activation='softmax'))

# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])  # compile the model

# model.fit(X_train, y_train, batch_size=128, epochs=20)  # fit the model

In [60]:
# model.save('/content/gdrive/MyDrive/MyBestModel2')

Extract the information from the board for each image and compute the results.

In [61]:
def extract_letters_cnn(image, patches):
    """
    Returns the matrix that encodes the image using a pretrained model.

    Parameters
    ----------
    image : numpy array
        The image to be processed
    patches : list
        The list of all patches

    Returns
    -------
    template: list of lists
       The matrix that encodes the image.
    """
    template = get_board_template()
    all_patches_to_decode = []
    indx = -1
    for i in range(NO_SQUARES):
        for j in range(NO_SQUARES):
            indx += 1
            # get the patch and preprocess it
            [(x1, y1), (x2, y2)] = patches[indx]
            cropped_image = image[y1:y2, x1:x2]
            cropped_image = cropped_image.astype(np.uint8)
            cropped_image = cv.cvtColor(cropped_image, cv.COLOR_BGR2GRAY)
            cropped_image = cv.GaussianBlur(cropped_image, (3, 3), 0)
            cropped_image = cv.GaussianBlur(cropped_image, (3, 3), 0)
            cropped_image = cv.GaussianBlur(cropped_image, (3, 3), 0)
            all_patches_to_decode.append(cropped_image)

    # use the model to predict the content of each patch
    all_patches_to_decode = np.array(all_patches_to_decode)
    all_patches_to_decode = all_patches_to_decode.reshape(all_patches_to_decode.shape[0], 150, 140, 1)
    all_patches_to_decode = all_patches_to_decode.astype('float32')
    all_patches_to_decode /= 255
    y_pred = model_cnn.predict(all_patches_to_decode)

    # transform the predictions back into our initial labels
    predictions = []
    for pred in y_pred:
        pred_flag = False
        for i, y in enumerate(pred):
            if y > 0.5:
                pred_flag = True
                predictions.append(i)
        if pred_flag == False:
            predictions.append(27)

    # add to the template the letters that were found
    indx = -1
    for i in range(NO_SQUARES):
        for j in range(NO_SQUARES):
            indx += 1
            letter = label_to_cat[predictions[indx]]
            if letter in ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']:
                template[i][j] = letter
            elif letter in ['NoLetter']:
                template[i][j] = '-'
    return template

Write the output of each image in a folder saved on google drive (/content/gdrive/MyDrive/results).

In [67]:
def write_output(score, new_letters, num_game, num_round):
    """
    Writes the output of each image into a text file.

    Parameters
    ----------
    score : float
        The score computed from the image
    new_letters : list
        The list of the new letters and their positions
    num_game : int
        The number of the game (from 1 to 5)
    num_round : int
        The number of the num_round (from 1 to 20)    

    Returns
    ----------
    """
    if not os.path.exists('/content/gdrive/MyDrive/results'):
        os.makedirs('/content/gdrive/MyDrive/results')
    str_num_round = ''
    if num_round < 10:
        str_num_round = '0' + str(num_round)
    else:
        str_num_round = str(num_round)
    f = open('/content/gdrive/MyDrive/results/' + str(num_game) + '_' + str_num_round + '.txt', 'w')
    for [l, i, j] in new_letters:
        f.write(str(i) + str(j) + ' ' + l + '\n')
    f.write(str(score))
    f.close()

# Run the solution to obtain the result

Unzip the CNN model and load it, then for each image in the dataset, read it, allign it with the template and crop it according to the template, extract the information from each patch and compute the score and the new letters (and their positions) from the board. Then, write the output into a corresponding output file and continue with the next picture.

Beacause we need the previous board to compute the score of the current board, we will keep track of it as well (at the beginning of each game, the previous board is just the template board).

In [ ]:
!unzip MyBestModel2.zip

In [65]:
# model_cnn = load_model('/content/gdrive/MyDrive/MyBestModel2')  # load the saved CNN model for patches recognition
model_cnn = load_model('MyBestModel2')

In [66]:
# read all images from train and add them in images_train array; for testing dataset just change train with test in BASE_FOLDER (defined at the beggining of this notebook)
base_folder = BASE_FOLDER
image_names = os.listdir(base_folder)
image_names.sort()
num_images = len(image_names) // 2
images_train = []
for img in image_names:
    if img[-3:] == 'jpg':
        image_now = cv.imread(os.path.join(base_folder, img))
        images_train.append(image_now)

In [68]:
# compute the output for each image
num_img = -1
prev_board = get_board_template()  # the board from the last round
curr_board = get_board_template()  # the board from the current round
for num_game in range(5):
    prev_board = get_board_template()  # at the beginning of each game, the board must be empty
    for num_round in range(20):
        num_img += 1
        img = images_train[num_img]  # get the current image
        image_now = cv.imread('template.jpg')  # get the template image
        image_then = img
        stitched = stitch_images_inside(image_source=image_then, image_dest=image_now)  # allign the image with the template
        image_then = cv.imread('template.jpg')
        image_now = stitched.astype(np.uint8)
        stitched2 = crop_image_as_template(image_source=image_then, image_dest=image_now)  # crop the image according to the template
        curr_board = extract_letters_cnn(stitched2, get_all_patches_from_template(335, 155))  # extract the information from each patch
        score, new_letters = board_score(prev_board, curr_board)  # compute the score and the new letters (and their positions) from the board
        write_output(score, new_letters, num_game + 1, num_round + 1)  # write the output into a corresponding output file
        prev_board = curr_board  # the current board will be the previous board in the next round

Compute the final score by comparing the files from the predictions with the files from the ground truth.

In [69]:
def compare_annotations(filename_predicted,filename_gt):
    """
    Computes the final score of the assignment.
    Code addapted from the assignment evaluation code.

    Parameters
    ----------
    filename_predicted : str
        The name of the filename that contains the predictions
    filename_gt : str
        The name of the filename that contains the ground truth 

    Returns
    ----------
    points_positions, points_letters, points_score : float
        The 3 scores obtained with the photo
    """
    p = open(filename_predicted,"rt")  	
    gt = open(filename_gt,"rt")  	
    all_lines_p = p.readlines()
    all_lines_gt = gt.readlines()

    number_lines_p = len(all_lines_p)
    number_lines_gt = len(all_lines_gt)

    match_positions = 1
    match_letters = 1
    match_score = 1

    if number_lines_gt != number_lines_p:  # added by me, because the code fails if the number of identified letters is diffrent from gt
        match_positions = 0
        match_letters = 0
    else:
        for i in range(number_lines_gt-1):
          current_pos_p, current_letter_p = all_lines_p[i].split()
          current_pos_gt, current_letter_gt = all_lines_gt[i].split()
          if(current_pos_p != current_pos_gt):
            match_positions = 0
          if(current_letter_p != current_letter_gt):
            match_letters = 0
    
    points_positions = 0.025 * match_positions
    points_letters = 0.015 * match_letters	

    #scores
    last_line_p = all_lines_p[-1]
    score_p = last_line_p.split()
    last_line_gt= all_lines_gt[-1]
    score_gt = last_line_gt.split()
    if(score_p != score_gt):
      match_score = 0

    points_score = 0.01 * match_score

    return points_positions, points_letters,points_score




predictions_path_root = "/content/gdrive/MyDrive/results/"
gt_path_root = "test/"

total_points = 0
for game in range(1,6):
	for turn in range(1,21):
		
		name_turn = str(turn)
		if(turn< 10):
			name_turn = '0'+str(turn)

		filename_predicted = predictions_path_root + str(game) + '_' + name_turn + '.txt'
		filename_gt = gt_path_root + str(game) + '_' + name_turn + '.txt'
		points_position, points_letters, points_score = compare_annotations(filename_predicted,filename_gt)
		total_points = total_points + points_position + points_letters + points_score

print(total_points)

2.5649999999999964
